In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np

In [3]:
abbr = 'SD'

In [4]:
file = 'south_dakota2018.xlsx'

Read in federal level data

In [5]:
fiscal = pd.read_sas('../../data/fiscal2018', format = 'sas7bdat', encoding='iso-8859-1')

Generate list of districts in the state in the federal data

In [6]:
state_fiscal = fiscal[(fiscal['STABBR'] == abbr) & (fiscal['GSHI'] == '12')]

In [7]:
len(state_fiscal)

156

Read in state level data

In [8]:
state_grads = pd.read_excel('../../data/state_data_raw/' + file, sheet_name='1718 HS Expanded')

In [9]:
state_grads.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,School Performance Index,Unnamed: 8,Unnamed: 9,...,Unnamed: 21,High School Completion Indicator,Unnamed: 23,Unnamed: 24,Unnamed: 25,College and Career Readiness,Unnamed: 27,Unnamed: 28,English Language Learners Progress,Unnamed: 30
0,District number,District name,School number,Entity ID,School name,School classification,Title I designation,Student Performance Total Points,High School Completion Indicator Total Points,College and Career Readiness Total Points,...,Math Percentage Level 3 and Level 4,High School Completion Student Count,High School Completion Rate Percentage,On-Time Graduation N-Size,On-Time Graduation Rate Percentage,Assessment Ready Percentage,Coursework Ready Percentage,Combined Ready Percentage,On Track Percentage of English Learners,Exited Percentage of English Learners
1,06001,Aberdeen 06-1,1,06001-01,Central High School - 01,Targeted Support,Non-Title I,31.40,22.57,13.18,...,0.3770,324,0.9198,324,0.8858,0.4678,0.5864,0.3627,6.78,0.0169
2,58003,Agar-Blunt-Onida 58-3,7,58003-07,Sully Buttes High School - 07,General Support,Non-Title I,39.35,27.27,25.97,...,0.3750,18,1,15,0.9333,0.9167,0.9167,0.8750,*,*
3,61001,Alcester-Hudson 61-1,1,61001-01,Alcester-Hudson High School - 01,General Support,Non-Title I,33.26,26.47,19.83,...,0.2917,23,0.9565,24,0.9167,0.5333,0.8667,0.5333,*,*
4,11001,Andes Central 11-1,1,11001-01,Andes Central High School - 01,Comprehensive Support,Title I,23,14.68,9.11,...,0.1538,20,0.6500,29,0.4138,0.2143,0.4286,0.1429,*,*


Reset columns.

In [10]:
state_grads.columns = state_grads.loc[0]
state_grads = state_grads.loc[1:]

Filter results.

In [11]:
state_grads = state_grads.dropna(how='all')

Select and rename columns.

In [12]:
state_grads = state_grads[['District name', 
                           'On-Time Graduation N-Size',
                           'On-Time Graduation Rate Percentage']]

In [13]:
state_grads.columns = ['District Name', 'Numerator', 'Graduation Rate']

In [14]:
state_grads.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 154 entries, 1 to 154
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   District Name    154 non-null    object
 1   Numerator        154 non-null    object
 2   Graduation Rate  154 non-null    object
dtypes: object(3)
memory usage: 4.8+ KB


Convert data types.

In [15]:
state_grads['Graduation Rate'] = state_grads['Graduation Rate'].astype(str).str.replace('%', '')
state_grads['Graduation Rate'] = state_grads['Graduation Rate'].astype(str).str.replace('\*', '')

state_grads['Numerator'] = state_grads['Numerator'].astype(str).str.replace('\*', '')

In [16]:
state_grads['District Name'] = state_grads['District Name'].astype(str)
state_grads['Numerator'] = pd.to_numeric(state_grads['Numerator'])
state_grads['Graduation Rate'] = pd.to_numeric(state_grads['Graduation Rate'])

Calculate class size.

In [17]:
state_grads['Total'] = np.round(state_grads['Numerator'] * (1 / state_grads['Graduation Rate']))

In [18]:
state_grads = state_grads[['District Name', 'Total', 'Graduation Rate']]

Check for matches and non-matches in the two lists. 

In [19]:
# state_fiscal['NAME'] = state_fiscal['NAME'].astype(str).str.upper().str.strip()
# state_grads['District Name'] = state_grads['District Name'].astype(str).str.upper().str.strip()

In [21]:
state_fiscal['NAME'] = state_fiscal['NAME'].astype(str).str.replace(' School District', '')
#state_fiscal['NAME'] = state_fiscal['NAME'].astype(str).str.replace(' ', ' ')

# state_grads['District Name'] = state_grads['District Name'].astype(str).str.replace(r' 1$', '')

In [22]:
matches = [name for name in list(state_grads['District Name']) if name in list(state_fiscal['NAME'])]
matches.sort()
len(matches)

149

In [23]:
A = [name for name in list(state_grads['District Name']) if name not in list(state_fiscal['NAME'])]
A.sort()
A

['Hitchcock-Tulare 56-6',
 'Irene-Wakonda 13-3',
 'Oldham-Ramona 39-5',
 'Platte-Geddes 11-5',
 'Wolsey-Wessington 02-6']

In [24]:
B = [name for name in list(state_fiscal['NAME']) if name not in list(state_grads['District Name'])]
B.sort()
B

['Big Stone City 25-1',
 'Black Hills Special Services Cooperative',
 'CORE Educational Cooperative',
 'East Dakota Educational Cooperative',
 'Elk Mountain 16-2',
 'Hitchcock Tulare 56-6',
 'Irene - Wakonda 13-3',
 'NORTHEAST TECHNICAL HIGH SCHOOL',
 'Northwest Area Schools ED Cooperative',
 'Oglala Lakota County 65-1',
 'Oldham - Ramona 39-5',
 'Platte - Geddes 11-5',
 'SD SCH FOR THE BLIND & VISUALLY IMPAIRED',
 'Southeast Area Cooperative',
 'Wolsey Wessington 02-6']

Make any additional matches I can find.

In [25]:
state_fiscal_rename = {
    'Hitchcock-Tulare 56-6' : 'Hitchcock Tulare 56-6',
    'Irene-Wakonda 13-3' : 'Irene - Wakonda 13-3',
    'Oldham-Ramona 39-5' : 'Oldham - Ramona 39-5',
    'Platte-Geddes 11-5' : 'Platte - Geddes 11-5',
    'Wolsey-Wessington 02-6' : 'Wolsey Wessington 02-6'
}

In [26]:
state_fiscal = state_fiscal.replace(state_fiscal_rename)

Merge federal and state data, keeping only matches between the two. 

In [27]:
state_grads_merged = pd.merge(state_fiscal, state_grads, how='inner', left_on='NAME', right_on='District Name')

Save cleaned data. 

In [28]:
state_grads_merged.to_csv('../../data/state_data_merged/' + abbr + '.csv', index=False)